In [5]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 7.8MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=166c8a25d6c09e50fbd496ae90330d6f17f6cca357d2a2c1db1ba99054fe1abf
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=fff7786e3aeb323dfad4aaa0f187ae1b40f4030a78fd32a9e53968c59f1f618d
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [6]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [10]:
fashion_mnist=keras.datasets.fashion_mnist

In [16]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [17]:
train_images=train_images/255.0
test_images=test_images/255.0

In [18]:
train_images[0].shape

(28, 28)

In [19]:
train_images=train_images.reshape(len(train_images),28,28,1)
train_images=test_images.reshape(len(test_images),28,28,1)

In [55]:
def build_model(hp):
  model=keras.Sequential([
        keras.layers.Conv2D(
            filters=hp.Int('conv_1_filter',min_value=32,max_value=128,step=16),
            kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
            activation='relu',
            input_shape=(28,28,1)
        ),
        keras.layers.Conv2D(
            filters=hp.Int('conv_2_filter',min_value=32,max_value=64,step=16),
            kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),
            activation='relu'
        ),
        keras.layers.Flatten(),
        keras.layers.Dense(
            units=hp.Int('dense_1_units',min_value=32,max_value=128,step=16),
            activation='relu'
        ),
        keras.layers.Dense(10,activation='softmax')#output layers
                
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

  return model

In [56]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [58]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [59]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 02m 22s]
val_accuracy: 0.09700000286102295

Best val_accuracy So Far: 0.10400000214576721
Total elapsed time: 00h 12m 53s
INFO:tensorflow:Oracle triggered exit


In [61]:
model=tuner_search.get_best_models(num_models=1)[0]

In [62]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 112)       2912      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 48)        48432     
_________________________________________________________________
flatten (Flatten)            (None, 23232)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                1486912   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                650       
Total params: 1,538,906
Trainable params: 1,538,906
Non-trainable params: 0
_________________________________________________________________
